# **결과**

## preprocssing0 + CNN_BiLSTM -> 제출 시: 0.50141 / - loss: 0.6147 - accuracy: 0.8757 

# **데이터 전처리0 & 임베딩**

In [ ]:
import pandas as pd

train = pd.read_csv("/content/drive/MyDrive/dacon 기후기술분류 ai 데이터/train.csv")
labels = pd.read_csv("/content/drive/MyDrive/dacon 기후기술분류 ai 데이터/labels_mapping.csv")

In [ ]:
test = pd.read_csv("/content/drive/MyDrive/dacon 기후기술분류 ai 데이터/test.csv")

0. 우선, 제출년도, 사업명, 사업_부처명, 계속과제여부, 내역사업명, 과제명 컬럼을 합쳐 전처리를 시도해보려고 한다.(null값이 존재하지 않는 컬럼들)

komoran의 형태소 추출 함수를 쓰려고 한다.

제출년도, 사업명, 사업_부처명, 계속과제여부, 내역사업명 컬럼 같은 경우는 거의 불용어 같은 것이 없으므로 불용어 제거, 토큰나이징에는 참여하지 않는다.

In [ ]:
import re
def preprocessing(text, komoran, remove_stopwords=False, stop_words=[]):
  #년도 때문에 0-9 포함
  #영어는 보통 그냥 단어(명사)인 경우로 명시되는 경우가 많아 그냥 영어는 전처리 안 하고 그대로 쓰도록 한다.
  text=re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ0-9a-zA-Z]","", text)
  if komoran=='okt':
    word_text=komoran.morphs(text,stem=True)
  else:
    word_text=komoran.morphs(text)
  if remove_stopwords:
    word_review=[token for token in word_text if not token in stop_words]
  return word_review

In [ ]:
pip install konlpy

     |████████████████████████████████| 19.4 MB 1.4 MB/s 
     |████████████████████████████████| 448 kB 57.7 MB/s 
     |████████████████████████████████| 86 kB 5.2 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
from konlpy.tag import Komoran
from konlpy.tag import Okt
#전처리 과정에서 모든 label에 공통적으로 쓰이는 단어도 포함
stop_words=['를','활용한','은','에','는','이','가', '하','아','것','들','의','있','되','수','보','주','등','한','위한','및','통한','개발','이용한','기술','연구','활용한','기반','기반의','미치는','개발을','대한','의한','관한']
komoran=Komoran()
okt=Okt()
train_text=[]

In [ ]:
import tqdm

for i in tqdm.tqdm(range(len(train))):
  try:
    #제출년도+사업명+사업_부서명+계속과제여부+내역사업명+과제명
    text = str(train.iloc[i,1])+" "+train.iloc[i,2]+" "+train.iloc[i,3]+" "+train.iloc[i,4]+" "+train.iloc[i,5]+" "+train.iloc[i,6]
    train_text.append(preprocessing(text, komoran, remove_stopwords=True, stop_words=stop_words))
  except:
    train_text.append([])

100%|██████████| 174304/174304 [04:41<00:00, 620.07it/s]


In [ ]:
test_text=[]

In [ ]:
for i in tqdm.tqdm(range(len(test))):
  try:
    text= str(test.iloc[i,1])+" "+test.iloc[i,2]+" "+test.iloc[i,3]+" "+test.iloc[i,4]+" "+test.iloc[i,5]+" "+test.iloc[i,6]
    test_text.append(preprocessing(text, komoran, remove_stopwords=True, stop_words=stop_words))
  except:
    test_text.append([])

100%|██████████| 43576/43576 [01:15<00:00, 580.84it/s]


# **임베딩**

In [ ]:
len(train_text)

174304

In [ ]:
len(test_text)

43576

**1. CountVectorizer를 통한 임베딩**

In [ ]:
# from sklearn.feature_extraction.text import CountVectorizer

# #tokenizer 인자에는 list를 받아서 그대로 내보내는 함수를 넣어줍니다. 또한 소문자화를 하지 않도록 설정해야 에러가 나지 않습니다.
# vectorizer = CountVectorizer(tokenizer = lambda x: x, lowercase=False)
# train_features=vectorizer.fit_transform(train_text)
# test_features=vectorizer.transform(test_text)
# #test데이터에 fit_transform을 할 경우 data leakage에 해당합니다

In [ ]:
# def tokenize_text(text):
#     '''토큰화 하여 인덱스를 돌려주는 함수

#     Args:
#         text: 문서들을 원소로 하는 리스트

#     Return:
#         토큰화하고 그것들을 id값으로 반환하는 리스트
#     '''
#     return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text))

In [ ]:
# tokenized_text = [tokenize_text(text) for text in train_features ]
# #tokenized_test = [tokenize_text(test_txt) for test_txt in test_features]

In [ ]:
# train['train_text']=0
# train['train_text']=str(train_text)

In [ ]:
# train['train_text']

In [ ]:
# test['test_text']=0
# for i in range(len(test_text)):
#   test.loc[i,'test_text']=str(test_text[i])

**2. Bert의 토크나이저를 불러와 토크나이즈 하기**

In [ ]:
!pip install bert-for-tf2

     |████████████████████████████████| 41 kB 134 kB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-py3-none-any.whl size=30536 sha256=0c1738080c35e474d17aca8c3054b4f750f43c0d48d5b67734786f051da8f911
  Stored in directory: /root/.cache/pip/wheels/47/b6/e5/8c76ec779f54bc5c2f1b57d2200bb9c77616da83873e8acb53
  Created wheel for params-flow: filename=params_flow-0.8.2-py3-none-any.whl size=19471 sha256=038e95c4c26295f4710fa999534b335ba3ef1e76746687da67a65a915a0f27b7
  Stored in directory: /root/.cache/pip/wheels/0e/fc/d2/a44fff33af0f233d7def6e7de413006d57c10e10ad736fe8f5
  Created wheel for py-params: filename=py_params-0.10.2-py3-none-any.whl size=7910 sha256=836da5f62c27bb766b5379bb40ce3ebf810f60f0d5ead991c54c1409863c9a06
  Stored in directory: /root/.cache/pip/wheels/e1/11/67/33cc51bbee127cb8fb2ba549cd29109b2f22da43ddf9969716
Successfully built bert-for-tf2 params-flow py-params


In [ ]:
# Bert의 토크나이저를 불러와 이를 이용하여 토크나이즈할 수 있도록 함
import bert
import tensorflow_hub as hub

BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1", trainable=False)
vocab = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocab, to_lower_case)

In [ ]:
def tokenize_text(text,tokenzier):
  all_tokens=[]
  '''토큰화 하여 인덱스를 돌려주는 함수

  Args:
    text: 문서들을 원소로 하는 리스트

  Return:
      토큰화하고 그것들을 id값으로 반환하는 리스트
  '''
  for texts in text:
    tokens_a = tokenizer.tokenize(str(texts))
    one_token = tokenizer.convert_tokens_to_ids(["[CLS]"]+tokens_a+["[SEP]"])
    all_tokens.append(one_token)
  return np.array(all_tokens)

In [ ]:
#tokenized_train = [tokenize_text(text) for text in train_text]
tokenized_train = tokenize_text(train_text,tokenizer)
#tokenized_test = [tokenize_text(test_txt) for test_txt in test_text]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  from ipykernel import kernelapp as app


In [ ]:
tokenized_test = tokenize_text(test_text,tokenizer)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  from ipykernel import kernelapp as app


In [ ]:
def get_max_length(df):
    '''단어 기준으로 문서들 중 가장 긴 것의 길이를 반환하는 함수
    
    padding을 하기 위해 최대값을 찾을 수 있도록 개별 작성한 함수

    Args:
        df: 원본 데이터프레임을 입력받음

    Return:
        max_length: 문서들 중 최대값을 반환
    '''
    max_length = 0
    for row in df:
        if len(str(row).split(" ")) > max_length:
            max_length = len(str(row).split(" "))
    return max_length

In [ ]:
MAX_SEQ_LEN = get_max_length(train_text)

In [ ]:
from tensorflow.keras import preprocessing

train_x = preprocessing.sequence.pad_sequences(tokenized_train, maxlen=MAX_SEQ_LEN, padding='post')

test_x = preprocessing.sequence.pad_sequences(tokenized_test, maxlen=MAX_SEQ_LEN, padding='post' )
